In [1]:
import os
import subprocess
import re
from os.path import join
from io import StringIO
from collections import defaultdict

import sympy
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import psutil

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_half_model_v2 import write_dicke_half_model, err_dicke_half

from doit.tools import register_doit_as_IPython_magic, config_changed

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
qnet.init_printing()

In [4]:
register_doit_as_IPython_magic()

In [5]:
if not 'MODULEPATH' in os.environ:
    f = open(os.environ['MODULESHOME'] + "/init/.modulespath", "r")
    path = []
    for line in f.readlines():
        line = re.sub("#.*$", '', line)
        if line is not '':
            path.append(line)
    os.environ['MODULEPATH'] = ':'.join(path)

if 'LOADEDMODULES' not in os.environ:
    os.environ['LOADEDMODULES'] = ''


In [6]:
def module(*args):
    if isinstance(args[0], list):
        args = args[0]
    else:
        args = list(args)
    (output, error) = subprocess.Popen(
            ['/usr/bin/modulecmd', 'python'] +
            args, stdout=subprocess.PIPE).communicate()
    exec(output)


In [7]:
module('load', 'intel/xe2011')

In [8]:
module('load', 'mpi')

In [9]:
module('list')

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Using the doit package for generating run data

## action wrappers

In [10]:
def update_config(rf, lambda_a, iter_stop):
    config = QDYN.config.read_config_file(join(rf, 'config'))
    config['oct']['iter_stop'] = iter_stop
    for pulse_config in config['pulse']:
        pulse_config['oct_lambda_a'] = lambda_a
    QDYN.config.write_config(config, join(rf, 'config'))

In [11]:
def run_traj_oct(rf, n_trajs):
    """Asynchronously run optimization, storing the pid of the oct process"""
    env = os.environ.copy()
    env['OMP_NUM_THREADS'] = '1'
    cmd = ['mpirun', '-n', str(int(n_trajs)), 'qdyn_optimize',
           '--n-trajs=%s' % int(n_trajs), '--J_T=J_T_sm', '.']
    with open(join(rf, 'oct.log'), 'wb') as log_fh:
        proc = subprocess.Popen(
            cmd, env=env, stdout=log_fh, stderr=subprocess.STDOUT, cwd=rf)
        with open(join(rf, 'oct.pid'), 'w') as pid_fh:
            pid_fh.write(str(proc.pid))

In [12]:
def run_traj_prop(rf, n_trajs):
    """Run propagation"""
    env = os.environ.copy()
    env['OMP_NUM_THREADS'] = '1'
    cmd = ['mpirun', '-n', str(int(n_trajs)), 'qdyn_prop_traj',
           '--n-trajs=%s' % int(n_trajs), '--use-oct-pulses',
           '--write-final-state=state_final.dat', '.']
    with open(join(rf, 'prop.log'), 'wb') as log_fh:
        subprocess.call(
            cmd, env=env, stdout=log_fh, stderr=subprocess.STDOUT, cwd=rf)

In [13]:
def wait_for_oct(rf):
    """Wait until the OCT process with the given pid ends"""
    try:
        with open(join(rf, 'oct.pid')) as pid_fh:
            pid = int(pid_fh.read())
            proc = psutil.Process(pid)
            proc.wait()
    except (psutil.NoSuchProcess, OSError):
        pass

## custom uptodate routines

In [14]:
def pulses_uptodate(rf, verbose=False):
    """Return True if all optimized pulses in the given runfolder
    are converged or have reached the max number of iterations, False
    otherwise"""
    config = QDYN.config.read_config_file(join(rf, 'config'))
    for pulse_config in config['pulse']:
        pulse_oct_file = join(rf, pulse_config['oct_outfile'])
        try:
            p = QDYN.pulse.Pulse.read(pulse_oct_file)
        except OSError:
            if verbose:
                print("    pulse %s does not exist" % pulse_oct_file)
            return False
        if p.oct_iter < config['oct']['iter_stop']:
            m = re.search(r'J_T\s*=\s*([0-9E.+-]+)', "".join(p.preamble))
            if verbose:
                print("    pulse %s has too foo iterations %d"
                      % (pulse_oct_file, p.oct_iter))
            if float(m.group(1)) > config['oct']['J_T_conv']:
                if verbose:
                    print("    pulse %s is not converged" % pulse_oct_file)
                return False
            else:
                if verbose:
                    print("    pulse %s is converged" % pulse_oct_file)
        if verbose:
            print("    pulse %s is up to date" % pulse_oct_file)
    return True

## task definitions

In [15]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    for ind, row in params_df.iterrows():
        rf = './data/doit/rf%d' % row['T']
        if rf in jobs:
            continue
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (write_dicke_half_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=True, non_herm=True,
                    lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'targets': [join(rf, 'config')],
            'uptodate': [True, ] # up to date if target exists
        }
    for job in jobs.values():
        yield job

In [16]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = './data/doit/rf%d' % row['T']
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [17]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = './data/doit/rf%d' % row['T']
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (run_traj_oct, [rf, ], dict(n_trajs=row['n_trajs']))],
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [18]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = './data/doit/rf%d' % row['T']
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_oct, [rf, ], {}),
                'rm -f %s' % join(rf, 'oct.pid')]}

In [19]:
def task_propagate():
    """Run optimization for every runfolder from params_df"""
    jobs = {}
    for ind, row in params_df.iterrows():
        rf = './data/doit/rf%d' % row['T']
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (run_traj_prop, [rf, ], dict(n_trajs=row['n_trajs']))],
            'file_dep': [join(rf, 'pulse1.oct.dat'),],}
    for job in jobs.values():
        yield job

##  Bringing it all together

In [20]:
params_data_str = r'''
#    T  lambda_a  n_trajs   iter_stop
    10     0.001       10          10
    10    0.0005       10          15
    10    0.0001       10          30
    20     0.001       10          20
    50     0.001       10          30
    70     0.001       10          30
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['T', 'lambda_a', 'n_trajs', 'iter_stop'])

In [21]:
params_df

,T,lambda_a,n_trajs,iter_stop
0,10,0.0010,10,10
1,10,0.0005,10,15
2,10,0.0001,10,30
3,20,0.0010,10,20
4,50,0.0010,10,30
5,70,0.0010,10,30


In [22]:
slh = network_slh(n_cavity=2, n_nodes=4, topology='driven_bs_fb')

In [23]:
%doit wait_for_optimization

-- create_runfolder:./data/doit/rf10
-- create_runfolder:./data/doit/rf20
-- create_runfolder:./data/doit/rf50
-- create_runfolder:./data/doit/rf70
.  update_runfolder:0
.  update_runfolder:3
.  update_runfolder:4
.  update_runfolder:5
-- submit_optimization:0
-- submit_optimization:3
-- submit_optimization:4
-- submit_optimization:5
.  wait_for_optimization:0
.  wait_for_optimization:3
.  wait_for_optimization:4
.  wait_for_optimization:5
.  update_runfolder:1
-- submit_optimization:1
.  wait_for_optimization:1
.  update_runfolder:2
-- submit_optimization:2
.  wait_for_optimization:2


In [24]:
%doit propagate

-- propagate:./data/doit/rf10
-- propagate:./data/doit/rf20
-- propagate:./data/doit/rf50
-- propagate:./data/doit/rf70
